In [1]:
import sys
assert sys.version_info >= (3, 5)

import numpy as np
import pandas as pd
import os

import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import model_from_json
assert tf.__version__ >= "2.0"

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from functools import partial

import winsound

In [2]:
f = 2000
t = 500

In [3]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [6]:
def prep_x(x_data):
    x_mean = x_data.mean(axis = 0, keepdims = True)
    x_std = x_data.std(axis = 0, keepdims = True) + 1e-7
    x_data = (x_data - x_mean)/x_std
    x_data = x_data[..., np.newaxis]
    return x_data

In [7]:
x_train = train_data.iloc[:, 1:]
x_train = x_train.values.astype(np.float32).reshape((x_train.shape[0], 28, 28)) / 255
x_valid, x_train = x_train[:4200], x_train[4200:]

y_train = train_data.iloc[:, 0]
y_train = y_train.values.astype(np.float32)
y_valid, y_train = y_train[:4200], y_train[4200:]

x_test = test_data.iloc[:, :]
x_test = test_data.values.astype(np.float32).reshape((x_test.shape[0], 28, 28)) / 255

x_train = prep_x(x_train)
x_valid = prep_x(x_valid)
x_test = prep_x(x_test)

In [8]:
print('x_train:')
print('Shape: ', x_train.shape)
print('Datatype: ', x_train.dtype)
print('Size: ', x_train.size)
print('Dimensions: ', x_train.ndim)

x_train:
Shape:  (37800, 28, 28, 1)
Datatype:  float32
Size:  29635200
Dimensions:  4


In [9]:
print('x_test:')
print('Shape: ', x_test.shape)
print('Datatype: ', x_test.dtype)
print('Size: ', x_test.size)
print('Dimensions: ', x_test.ndim)

x_test:
Shape:  (28000, 28, 28, 1)
Datatype:  float32
Size:  21952000
Dimensions:  4


In [10]:
#def_conv2d = partial(keras.layers.Conv2D, kernel_size = 3, activation = 'relu', padding = "SAME")

In [11]:
#cnn = keras.models.Sequential([
#    def_conv2d(filters = 64, kernel_size = 7, input_shape = [28, 28, 1]),
#    keras.layers.MaxPooling2D(pool_size = 2),
#    def_conv2d(filters = 128),
#    def_conv2d(filters = 128),
#    keras.layers.MaxPooling2D(pool_size = 2),
#    def_conv2d(filters = 256),
#    def_conv2d(filters = 256),
#    keras.layers.MaxPooling2D(pool_size = 2),
#    keras.layers.Flatten(),
#    keras.layers.Dense(units = 128, activation = 'relu'),
#    keras.layers.Dropout(0.5),
#    keras.layers.Dense(units = 64, activation = 'relu'),
#    keras.layers.Dropout(0.5),
#    keras.layers.Dense(units = 10, activation = 'softmax')
#])

In [12]:
#cnn.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
#history = cnn.fit(x_train, y_train, epochs = 10, validation_data = [x_valid, y_valid])

#cnn_json = cnn.to_json()
#with open("cnn.json", "w") as json_file:
#    json_file.write(cnn_json)
    
#cnn.save_weights("cnn.h5")
#print("CNN has been saved.")

#winsound.Beep(f, t)

In [13]:
json_file = open('cnn.json', 'r')
cnn_load_json = json_file.read()
json_file.close()
cnn_load = model_from_json(cnn_load_json)
cnn_load.load_weights("cnn.h5")
print("CNN has been loaded.")

cnn_load.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
y_pred_cnn = cnn_load.predict(x_test)
winsound.Beep(f, t)

CNN has been loaded.


In [14]:
cnn_load.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        3200      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 256)         5

In [15]:
print('y_pred_cnn:')
print('Shape: ', y_pred_cnn.shape)
print('Datatype: ', y_pred_cnn.dtype)
print('Size: ', y_pred_cnn.size)
print('Dimensions: ', y_pred_cnn.ndim)
y_pred_cnn[0:3]

y_pred_cnn:
Shape:  (28000, 10)
Datatype:  float32
Size:  280000
Dimensions:  2


array([[0.0000000e+00, 8.9150226e-35, 1.0000000e+00, 6.1040491e-30,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.8151322e-34,
        0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 1.0387188e-35, 1.4622281e-33, 1.4577692e-30,
        9.5489438e-32, 2.3885880e-27, 2.8687649e-17, 0.0000000e+00,
        1.8747126e-24, 6.1232959e-28],
       [1.9546226e-18, 1.1088184e-18, 8.9207318e-18, 2.0381571e-11,
        1.7236451e-06, 2.6558950e-12, 1.2597572e-19, 1.3845497e-11,
        7.8292851e-11, 9.9999833e-01]], dtype=float32)

In [16]:
y_pred_cnn_sub = np.argmax(y_pred_cnn, axis = 1)
y_pred_cnn_sub

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [17]:
y_pred_i = np.arange(1, len(y_pred_cnn_sub) + 1)

In [18]:
submission = pd.DataFrame(y_pred_i)
submission = pd.concat([submission, pd.DataFrame(y_pred_cnn_sub)], axis = 1)
submission.columns = ["ImageId", "Label"]
submission.to_csv("sub_cnn_01.csv", index = False)

In [19]:
def pad_x(x_data):
    x_data = np.pad(x_data, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
    return x_data

In [20]:
x_train_pad = pad_x(x_train)
x_valid_pad = pad_x(x_valid)
x_test_pad = pad_x(x_test)

In [21]:
print('x_train_pad:')
print('Shape: ', x_train_pad.shape)
print('Datatype: ', x_train_pad.dtype)
print('Size: ', x_train_pad.size)
print('Dimensions: ', x_train_pad.ndim)

x_train_pad:
Shape:  (37800, 32, 32, 1)
Datatype:  float32
Size:  38707200
Dimensions:  4


In [22]:
lenet5 = keras.models.Sequential([
    keras.layers.Conv2D(filters = 6, kernel_size = (3, 3), activation = 'relu', input_shape = (32, 32, 1)),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu'),
    keras.layers.AveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 120, activation = 'relu'),
    keras.layers.Dense(units = 84, activation = 'relu'),
    keras.layers.Dense(units = 10, activation = 'softmax'),
])

In [23]:
lenet5.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 6)         60        
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
average_pooling2d_1 (Average (None, 6, 6, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               69240     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [24]:
#cnn.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
#history = cnn.fit(x_train, y_train, epochs = 10, validation_data = [x_valid, y_valid])

#cnn_json = cnn.to_json()
#with open("cnn.json", "w") as json_file:
#    json_file.write(cnn_json)
    
#cnn.save_weights("cnn.h5")
#print("CNN has been saved.")

#winsound.Beep(f, t)

In [25]:
#json_file = open('cnn.json', 'r')
#cnn_load_json = json_file.read()
#json_file.close()
#cnn_load = model_from_json(cnn_load_json)
#cnn_load.load_weights("cnn.h5")
#print("CNN has been loaded.")

#cnn_load.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
#y_pred_cnn = cnn_load.predict(x_test)
#winsound.Beep(f, t)